In [1]:
import pandas as pd

In [76]:
questions = pd.read_csv("questions.csv")
paragraphs = pd.read_csv("paragraphs.csv")
sample = pd.read_csv("sample.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [82]:
# Test data
# It pairs all test question ids with all available paragraphs.
# 
# A question-paragraph id is generated by concatenating the id of the question to the symbol “#”
# and then to the paragraph id.
# 
# You will need to parse it to extract the two identifiers (e.g. question-id and paragraph-id). 
# 
# Format:
# id – identifier for question-paragraph pair 
# qpid – (question-id # para-id) – combined id or question and paragraph

test.head()

,id,qpid,qid,ParaId
0,0,1235#0,1235,0
1,1,1235#1,1235,1
2,2,1235#2,1235,2
3,3,1235#3,1235,3
4,4,1235#4,1235,4


In [78]:
# This is your training data, showing the paragraphs that are relevant for the particular question.
# 
# Format:
# qid – id of question (from questions.csv)
# ParaId – paragraph id for correct answer (from paragraphs.csv)

train.head()

,qid,ParaId
0,580,21
1,1188,323
2,1072,226
3,15,499
4,589,28


In [40]:
# This is the what result of your model should look like and is the file that should be submitted for evaluation. 
# 
# Format:
# id – identifier for question-paragraph pair (from test.csv)
# target (provided by your model) – 1 if para-id is in the top 5 relevant paragraphs for question q-id, 0 otherwise

sample.head()

,id,target
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [31]:
# One paragraph can be the answer to multiple questions

train.groupby('ParaId').count().head()

,qid
ParaId,
2,1
3,1
5,6
6,1
7,4


In [81]:
# Split qpid into qid and ParaId

test['qid'], test['ParaId'] = test['qpid'].str.split('#', 1).str
test.head()

,id,qpid,qid,ParaId
0,0,1235#0,1235,0
1,1,1235#1,1235,1
2,2,1235#2,1235,2
3,3,1235#3,1235,3
4,4,1235#4,1235,4


In [9]:
test.head()

,id,qpid,qid,ParaId
0,0,1235#0,1235,0
1,1,1235#1,1235,1
2,2,1235#2,1235,2
3,3,1235#3,1235,3
4,4,1235#4,1235,4


In [12]:
test['target'] = test.name.isin(Df2.IDs).astype(int)
test.head()

,id,qpid,qid,ParaId
0,0,1235#0,1235,0
1,1,1235#1,1235,1
2,2,1235#2,1235,2
3,3,1235#3,1235,3
4,4,1235#4,1235,4


In [35]:
train['qpid'] = train['qid'].astype(str).str.cat(train['ParaId'].astype(str), sep='#')
train.head()

,qid,ParaId,qpid
0,580,21,580#21
1,1188,323,1188#323
2,1072,226,1072#226
3,15,499,15#499
4,589,28,589#28


In [79]:
test['is_qid_present_in_train'] = train['qid'].astype(int).isin(test['qid'].astype(int)).astype(int)

KeyError: 'qid'

In [74]:
test.is_qid_present_in_train.describe()

count    676.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: is_qid_present_in_train, dtype: float64

In [90]:
test[(test['qid'] == '15')]

SyntaxError: invalid syntax (<ipython-input-90-62a9233465f7>, line 1)

In [95]:
train[train["qid"].str.contains('1235')]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas